# Robodyno MiMotor API

## 1.电机状态设置

### 导入robodyno库，创建CAN总线对象

In [5]:
from robodyno.components.can_bus.mi_motor import MiMotor
from robodyno.interfaces import CanBus
can = CanBus()

### 搜索总线上所有的小米电机

In [ ]:
for id in range(0x01, 0x80):
    data = MiMotor.get_mcu_uuid(can, id , 0.01)
    if data is not None:
        print(f"Found motor with id: {id}, mcu_uuid: {data}")
        break
print("Done")


### 初始化小米电机对象

`MiMotor(can, id_=0x7F)`

参数：
- `can` : can总线接口
- `id_` : 电机ID,默认0x7F（ 范围从 0x01 到 0x7F ）

In [7]:
motor = MiMotor(can, 0x7F)

### 电机使能

`enable()`

- 使能后，可以控制电机。

In [20]:
motor.enable()

### 电机失能

`disable()`

- 这是上电后的默认状态。
- 失能后，电机停止工作，转矩松动。

In [3]:
motor.disable()

### 初始化当前位置

`init_pos()`

- 将电机的当前位置设置为初始位置。

In [5]:
motor.init_pos()

### 电机校准

`calibrate()`

- 在失能状态下运行。

In [ ]:
motor.calibrate()

### 设置电机CAN_ID

`config_can_bus(new_id)`

参数 ：

- `new_id` ：电机新CAN_ID (0x01~0x7F), 与标准robodyno库中的id不冲突

In [ ]:
motor.config_can_bus(new_id = 0x7F)

### 保存设置

`save()`
- 保存电机参数。

In [ ]:
motor.save()

### 清除错误

`clear_errors()`

- 如果解决了原因，大多数错误都会自动清除。

In [7]:
motor.clear_errors()

### 电机软急停

`estop()`

- 暂不支持该接口，将抛出一个NotImplementedError异常。

In [ ]:
motor.estop()

### 电机重启

`reboot()`

- 暂不支持该接口，将抛出一个NotImplementedError异常。

In [ ]:
motor.reboot()

### 恢复出厂设置

`reset()`

- 此命令将电机重置为出厂设置。所有配置将丢失，id将重置为`0x7F`。

In [6]:
motor.reset()

## 2.电机模式设置

### 进入MIT模式

`mit_mode()`

- 使用MIT模式控制电机的位速力

In [3]:
motor.mit_mode()

### 进入直接位置模式

`position_mode()`

- 直接使用PID控制位置

In [19]:
motor.position_mode()

### 进入直接速度模式

`velocity_mode()`

- 直接使用PID控制速度

In [3]:
motor.velocity_mode()

### 进入力矩控制模式

`torque_mode()`

- 设置电机为力矩控制

In [7]:
motor.torque_mode()

## 3.电机参数设置

### 设置MIT模式

`set_mit(torque, position, velocity, kp, kd)`

参数:
- `torque`: 力矩
- `position`: 位置
- `velocity`: 速度
- `kp`: 位置系数
- `kd`: 速度系数

In [5]:
torque = 0
position = 1
velocity = 0
kp = 1
kd = 0
motor.set_mit(torque, position, velocity, kp, kd)

### 设置位置

`set_pos(pos)`

参数:

- `pos`: 目标位置(rad)

In [21]:
motor.set_pos(-21.57)

### 设置速度

`set_vel(vel)`

参数：

- `vel`: 目标速度(rad/s)

In [4]:
motor.set_vel(1)

### 设置力矩, 电流再考虑
todo: 确定这里的是力矩还是电流

`set_torque(torque)`

参数：

- `torque`: 目标力矩(Nm)

In [13]:
motor.set_torque(-0.12)

### 设置电机PID参数

todo: pid 代码修改

`set_pid(pos_kp, vel_kp, vel_ki)`

参数:

- `pos_kp`: 位置环比例系数
- `vel_kp`: 速度环比例系数
- `vel_ki`: 速度环积分系数

In [52]:
motor.set_pid(2, 0.021, 30)

### 设置电机速度限制
在位置模式中生效

`set_vel_limit(vel_lim)`

参数：

- `vel_lim`: 输出端最大速度(rad/s), 必须为正值。

In [62]:
motor.set_vel_limit(3.14)

### 设置电机电流限制
在速度模式中生效

`set_current_limit(current_lim)`

参数:

- `current_lim` : 最大电流(A), 必须为正值。

In [64]:
motor.set_current_limit(10)

## 4.读取电机参数

### 读取电机详细状态

`get_state(timeout = 0.1)`

参数：

- `timeout` : 请求超时时间(s)，None代表无超时时间

返回值 ：

- 电机状态(MotorState)
    - 0-空闲，1-校准，2-使能
- 电机错误(error)
    - `error` : 错误码（1-电压不足，2-电流过大，4-温度过高，8-磁编码错误，16-霍尔编码错误，32-未校准）
    - `motor_err` : 直接返回0
    - `encoder_err` : 直接返回0
    - `controller_err` : 直接返回0
- 电机控制模式(MotorControlMode)
    - MIT模式 : 0
    - 位置模式 : 1
    - 速度模式 : 2
    - 力矩模式 : 3

In [ ]:
motor.get_state()

### 读取总线电压

`get_voltage(timeout = 0.1)`

参数：

- `timeout` : 请求超时时间(s)，None代表无超时时间

返回值 ：

- 总线电压值 ( V ) ，超时则不返回

In [ ]:
motor.get_voltage(1)

### 读取电机温度

`get_temperature(timeout = 0.1)`

参数：

- `timeout` : 请求超时时间(s)，None代表无超时时间

返回值：

- 电机温度 ( °C ) ，超时则不返回

In [ ]:
motor.get_temperature(1)

### 读取电机控制模式

`get_mode(timeout = 0.1)`

参数：

- `timeout` : 请求超时时间(s)，None代表无超时时间
返回值：

 - 控制模式(int)
    - 0 mit
    - 1 位置
    - 2 速度
    - 3 力矩

In [ ]:
motor.get_mode()

### 读取电机参数反馈

`get_feedback(timeout = 0.1)`

参数：

- `timeout` : 请求超时时间(s)，None代表无超时时间

返回值 ：

- 电机参数(位置rad, 速度rad/s, 力矩Nm)，超时则不返回

In [ ]:
motor.get_feedback(1)

### 读取电机位置

`get_pos(timeout = 0.1)`

参数：

- `timeout` : 请求超时时间(s)，None代表无超时时间

返回值：

- 位置(rad)，超时则不返回

In [ ]:
motor.get_pos(1)

### 读取电机速度

`get_vel(timeout = 0.1)`

参数：

- `timeout` : 请求超时时间(s)，None代表无超时时间

返回值：

- 速度(rad/s)，超时则不返回

In [ ]:
motor.get_vel(1)

### 读取电机力矩

`get_torque(timeout = 0.1)`

参数：

- `timeout` : 请求超时时间(s)，None代表无超时时间

返回值：

- 力矩(Nm)，超时则不返回

In [ ]:
motor.get_torque(1)

### 读取电机PID参数

todo: 读取pid 程序 修改

`get_pid(timeout = 0)`

参数：

- `timeout` : 请求超时时间(s)，0代表无超时时间

返回值：

- 电机PID
    - `pos_kp` : 位置环P
    - `vel_kp` : 速度环P
    - `vel_ki` : 速度环I

In [ ]:
motor.get_pid(1)

### 读取电机速度限制

`get_vel_limit(timeout = 0)`

参数:

- `timeout` : 请求超时时间(s)，0代表无超时时间

返回值：

- 输出端最大速度(rad/s)，超时则不返回

In [ ]:
motor.get_vel_limit()

### 读取电机电流限制

`get_current_limit(timeout = 0)`

参数:

- `timeout` : 请求超时时间(s)，0代表无超时时间

返回值：

- 最大电流(A)，超时则不返回

In [ ]:
motor.get_current_limit()

### 读取电机版本

`get_version(timeout = 0)`

参数：

- `timeout` : 请求超时时间(s)，0代表无超时时间

返回值 ：

- API版本dict
    - `main_version` : 主版本号
    - `sub_version` : 副版本号
    - `data` : 电机减速比
    - `type` : 设备类型
      - CYBER GEAR

In [ ]:
motor.get_version(1)